In [2]:
# import pandas as pd
# import inflect

# # Create an engine for inflect
# p = inflect.engine()

# def convert_numbers_to_words(text):
#     # Split the text into words
#     words = text.split()
#     # For each word in the text
#     for i in range(len(words)):
#         # If the word is a number
#         if words[i].isdigit():
#             # Convert the number to words and replace the number with words in the text
#             words[i] = p.number_to_words(words[i])
#     # Join the words back into a text string
#     return ' '.join(words)

# # Read the data from the CSV file
# df = pd.read_csv('./')

# # Apply the function to the 'pn_history' column
# df['pn_history'] = df['pn_history'].apply(convert_numbers_to_words)

# # Write the data back to the CSV file
# df.to_csv('data.csv', index=False)


In [7]:
import pandas as pd
import inflect
import re

# Create an engine for inflect
p = inflect.engine()

# Standardizing Formats "17", “17-year-old”, “3-4”, or “17yo”
def convert_numbers_in_string(s):
    # Find all numbers in the string
    numbers_in_words = re.findall(r'\b\d+\b', s)
    numbers_in_parts = re.findall(r'\d+', s)

    # For each number
    for number in numbers_in_words:
        # Convert the number to words
        word = p.number_to_words(number)
        # Replace the number with the word in the string
        s = re.sub(r'\b' + number + r'\b', word, s)

    for number in numbers_in_parts:
        # Convert the number to words
        word = p.number_to_words(number)
        # Replace the number with the word (followed by a space) in the string
        s = re.sub(number, word + " ", s)

    return s

# Handling Contractions: Expanding contractions" 
# "yo", "y.o." "y/o" to "year old",  
# "y" to "year"
# "f" to "female", 
# "m" to "male", 
# "mo" to "month", 
# "yr" to "year"
# "c/o" "c/of" "c/m" to "complains of",
# "cc" to "chief complaint"
# "h/o" to "history of"
# "pt" to "patient"
# "w", "wk" to "week", "hrs" to "hours""
# "x" to times?
def expand_contractions(text):
    # Define a dictionary of contractions and their expanded forms
    contractions = {
        "yo": "year old",
        "y.o.": "year old",
        "y/o": "year old",
        "y": "year",
        "f": "female",
        "m": "male",
        "mo": "month",
        "yr": "year",
        "c/o": "complains of",
        "c/of": "complains of",
        "c/m": "complains of",
        "cc": "chief complaint",
        "h/o": "history of",
        "pt": "patient",
        "w": "week",
        "wk": "week",
        "hrs": "hours"
    }
    # Split the text into words
    words = text.split()
    # For each word in the text
    for i in range(len(words)):
        # If the word is a contraction
        if words[i] in contractions:
            # Replace the contraction with its expanded form in the text
            words[i] = contractions[words[i]]
    # Join the words back into a text string
    return ' '.join(words)

# def expand_contractions(text):
#     # Define a dictionary of contractions and their expanded forms
#     contractions = {
#         r"\byo\b": "year old",
#         r"\by\.o\.\b": "year old",
#         r"\by/o\b": "year old",
#         r"\by\b": "year",
#         r"\bf\b": "female",
#         r"\bm\b": "male",
#         r"\bmo\b": "month",
#         r"\byr\b": "year",
#         r"\bc/o\b": "complains of",
#         r"\bc/of\b": "complains of",
#         r"\bc/m\b": "complains of",
#         r"\bcc\b": "chief complaint",
#         r"\bh/o\b": "history of",
#         r"\bpt\b": "patient",
#         r"\bw\b": "week",
#         r"\bwk\b": "week",
#         r"\bhrs\b": "hours"
#     }
#     # For each contraction in the dictionary
#     for contraction, expansion in contractions.items():
#         # Use regex to find the contraction in the text and replace it with its expanded form
#         text = re.sub(contraction, expansion, text)
#     return text


def process_data(df):
    # Fill NaN values with a default value
    df.fillna('Unknown', inplace=True)
    
    # Convert columns to string type
    df['pn_history'] = df['pn_history'].astype(str)

    df['pn_history'] = df['pn_history'].apply(convert_numbers_in_string)

    # Apply the function to the 'pn_history' column
    df['pn_history'] = df['pn_history'].apply(expand_contractions)

    return df

# Read the data from the CSV file
df = pd.read_csv('./patient_notes.csv')

# Apply the function to the DataFrame
df = process_data(df)

# Write the data back to the CSV file
df.to_csv('data.csv', index=False)
